In [1]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

# Langchain

prompt format 생성

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

1. github에서 글자 긁어오기
2. vector store 생성
3. langchain hub에서 프롬프트 생성
```
{
"input": "What is Task Decomposition?"
}
```
5. LLM 답변 확인

In [4]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# text split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps, often using techniques like Chain of Thought (CoT) to enhance reasoning and model performance. It allows for a structured approach to problem-solving by framing large tasks as a series of subgoals, which can be tackled sequentially. This method can be implemented using various prompting strategies, task-specific instructions, or human input.'

In [5]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### 1. 3개의 블로그 포스팅 본문을 Load하기 : WebBaseLoader 활용
```
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
```

In [6]:
def load(url): 
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("post-content", "post-title", "post-header")
            )
        ),
    )
    docs = loader.load()

    return docs[0]

In [7]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = []

for url in urls:
    docs.append(load(url))

### 2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용 (아래 링크 참고)
### 3. Chunks 를 임베딩하여 Vector store 저장: openai 임베딩, chroma vectorstore 사용

In [8]:
# 2 ~ 3
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

In [9]:
# 검색 테스트
results = vectorstore.similarity_search(
    "how to use CoT",
    k=6
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Chain-of-Thought (CoT)#
Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using large models (e.g. with more than 50B parameters). Simple tasks only benefit slightly from CoT prompting.
Types of CoT prompts#
Two main types of CoT prompting: [{'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'}]
* Few-shot CoT. It is to prompt the model with a few demonstrations, each containing manually written (or model-generated) high-quality reasoning chains. [{'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'}]
* Fig. 3. Comparing CoT and PoT. (Image source: Chen et al. 2022).
External APIs#
TALM (Tool Augmented Language Models; Parisi et al. 2022) is a language model augmented with text-to-text API calls

### 4. User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve

retriever search_type 은 'similarity', search_kwargs={'k': 6} 을 사용해주세요.

In [10]:
# 4.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
print(retriever.invoke("agent memory"))

[Document(id='e534f645-0f34-4da1-9c4b-a282e22dbbdf', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.\n\nRecency: recent events have higher scores\nImportance: distinguish mundane from core memories. Ask LM directly.\nRelevance: based on how related it is to the current situation / query.\n\n\nReflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection above)'), Document(id

### 5. 5-1) 과 5-2) 를 참고하여, User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트를 작성해보세요: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 )

5-1)

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [12]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

5-2)

In [13]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'joke': "Why don't skeletons fight each other? They don't have the guts!"}

In [14]:
prompt = """
유저의 question과 content가 들어오면, question과 content가 서로 관련이 있는지 알려줘.
관련이 있다면, yes
관련이 없다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
"""

In [30]:
def relevance_checker(question):
    prompt = """
유저의 question과 content가 들어오면, question과 content가 서로 관련이 있는지 알려줘.
관련이 있다면, yes
관련이 없다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
    """
    parser = JsonOutputParser()
    
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    retriver_result = retriever.invoke(question)
    query = "question: " + question + "\n\n"
    query += "content: " + str(retriver_result)
    
    prompt = PromptTemplate(
        template=prompt,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": query})
    return result, retriver_result

In [16]:
relevance_checker("agent memory")

({'related': 'yes'},
 [Document(id='e534f645-0f34-4da1-9c4b-a282e22dbbdf', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.\n\nRecency: recent events have higher scores\nImportance: distinguish mundane from core memories. Ask LM directly.\nRelevance: based on how related it is to the current situation / query.\n\n\nReflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection

## 6. 5 에서 모든 docs에 대해 'yes' 가 나와야 하는 케이스와 ‘no’ 가 나와야 하는 케이스를 작성해보세요.
## 7. 5에서 케이스별로 의도한 결과 ('yes' 또는 'no' )와 일치하는 답변이 나오는지 확인해보세요. 정답대로 나오지 않는다면 문제를 찾아 디버깅해보세요. (Splitter, Chunk size, overlap, embedding model, vector store, retrieval 평가 시스템 프롬프트 등) 디버깅이 어려운 경우, 강사/조교에게 질문해주세요.

In [17]:
relevance_checker("how to use CoT")

({'related': 'yes'},
 [Document(id='dec9ff83-50b3-481d-add5-35d48dab7042', metadata={'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'}, page_content='Chain-of-Thought (CoT)#\nChain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using large models (e.g. with more than 50B parameters). Simple tasks only benefit slightly from CoT prompting.\nTypes of CoT prompts#\nTwo main types of CoT prompting:'),
  Document(id='8c01a224-d811-4983-afb3-8914f97a8b40', metadata={'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'}, page_content='Few-shot CoT. It is to prompt the model with a few demonstrations, each containing manually written (or model-generated) high-quality reasoning chains.'),
  Document(id='31ce

In [18]:
relevance_checker("i like apple")

({'related': 'no'},
 [Document(id='87e924db-2f69-4b73-a261-8a757476ef58', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='API-Bank (Li et al. 2023) is a benchmark for evaluating the performance of tool-augmented LLMs. It contains 53 commonly used API tools, a complete tool-augmented LLM workflow, and 264 annotated dialogues that involve 568 API calls. The selection of APIs is quite diverse, including search engines, calculator, calendar queries, smart home control, schedule management, health data management, account authentication workflow and more. Because there are a large number of APIs, LLM first has access to API search engine to find the right API to call and then uses the corresponding documentation to make a call.'),
  Document(id='3cb6a4ff-2998-4b1e-9cdb-4f213dfb071a', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Maximum Inner Product Search (MIPS)#\nThe external memory can alleviate the 

In [19]:
relevance_checker("I hate AI")

({'related': 'no'},
 [Document(id='6f2fbbc6-3a38-45e2-8ba8-509ce318aa2f', metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='Fig. 13. UI for humans to do tool-assisted adversarial attack on a classifier. Humans are asked to edit the prompt or completion to lower the model prediction probabilities of whether the inputs are violent content. (Image source: Ziegler et al. 2022)'),
  Document(id='e25c0bc5-4134-409c-87a4-2565ccfaeb31', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 13. The generative agent architecture. (Image source: Park et al. 2023)\nThis fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g. host a party and invite many others).\nProof-of-Concept Examples#\nAutoGPT has drawn a lot of attention into the possibility of setting up autonomou

## 8. 6-7 의 평가에서 문제가 없다면, 5에서 작성한 코드의 실행 결과가 'yes' 인 경우, 4의 retrieved chunk 를 가지고 답변하는 chain 코드를 작성해주세요. (prompt | llm | parser 형태의 코드)

In [98]:
from openai import OpenAI

def generate_answer(question):
    parser = JsonOutputParser()
    
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
    retriver_result = retriever.invoke(question)
    query = "question: " + question + "\n\n"
    query += "content: " + str(retriver_result)
    
    prompt = PromptTemplate(
        template="""
유저의 question과 content가 들어오면, question과 content가 서로 관련이 있는지 알려줘.
관련이 있다면, yes
관련이 없다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
    """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": query})

    if result['related'] == 'yes':
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "content를 참고하여 question의 답변을 작성해줘"
                }, 
                {
                    "role": "user",
                    "content": query
                }
            ]
        )

        return response.choices[0].message.content, str(retriver_result)
    else:
        return result

In [99]:
generate_answer("agent memory")

("Agent memory refers to the mechanisms by which an autonomous agent retains, recalls, and utilizes information over time to inform its behavior. It can be categorized primarily into short-term and long-term memory:\n\n1. **Short-term memory**: This utilizes in-context learning, allowing the agent to draw insights and make decisions based on information present in the immediate context.\n\n2. **Long-term memory**: This aspect enables the agent to keep extensive records of experiences and knowledge over extended periods. It typically involves an external database or vector store which supports the retrieval of relevant information when needed.\n\nThe memory system in agent architecture also encompasses components like a retrieval model, which helps surface context based on factors such as recency (recent events are prioritized), importance (differentiating core memories from mundane ones), and relevance (how closely related memory is to the current task).\n\nFurthermore, there are refle

## 9. 생성된 답안에 Hallucination 이 있는지 평가하는 시스템 프롬프트를 작성해보세요. LLM이 스스로 평가하도록 하고, hallucination 이 있으면 {‘hallucination’: ‘yes’} 없으면 {‘hallucination’: ‘no’} 라고 출력하도록 하세요.

In [114]:
def hallucination_answer(answer, docs):
    parser = JsonOutputParser()
    
    query = "answer: " + answer + "\n\n"
    query += "docs: " + docs
    
    prompt = PromptTemplate(
        template="""
유저의 answer과 docs가 들어오면, docs에 근거하여 답변이 잘 작성되었는지 확인해줘.
docs에 없는 내용을 지어냈는지 확인해야해.
말을 지어냈다면,  yes
docs에 근거했다면, no
답변해줘

Answer the user query.\n{format_instructions}\n{query}\n
key 값은 hallucination으로 해줘
    """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = prompt | llm | parser
    
    result = chain.invoke({"query": query})

    return result

In [115]:
answer, docs = generate_answer("agent memory")

In [116]:
answer

"Agent memory can be categorized into two main types: short-term memory and long-term memory. \n\n- **Short-term memory** refers to in-context learning, where the model uses immediate information to adapt and respond during a single interaction. It allows the agent to leverage recent contextual cues without retaining them over time.\n\n- **Long-term memory**, on the other hand, enables the agent to retain and recall vast amounts of information over extended periods. This is often facilitated by an external memory module or vector store, which helps the agent access and retrieve relevant past experiences efficiently.\n\nIn addition to these memory types, an agent's system also incorporates a reflection mechanism that synthesizes past experiences into higher-level inferences, guiding future behavior based on learned lessons. The effectiveness of memory in this context is influenced by factors such as recency, importance, and relevance of past events to current tasks or queries. \n\nOvera

In [117]:
hallucination_answer(answer, docs)

{'hallucination': 'no'}

## 10. 9 에서 ‘yes’ 면 8 로 돌아가서 다시 생성, ‘no’ 면 답변 생성하고 유저에게 답변 생성에 사용된 출처와 함께 출력하도록 하세요. (최대 1번까지 다시 생성)

In [119]:
result = hallucination_answer(answer, docs)
result

{'hallucination': 'no'}

In [124]:
if result['hallucination'] == 'no':
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "content에서 source를 뽑아서, [answer 내용]\n source: [source]형태로 적어줘"
            }, 
            {
                "role": "user",
                "content": 'answer: ' + answer + '\ndocs:' + docs
            }
        ]
    )

    print(response.choices[0].message.content)

Agent memory can be categorized into two main types: short-term memory and long-term memory.

- **Short-term memory** refers to in-context learning, where the model uses immediate information to adapt and respond during a single interaction. It allows the agent to leverage recent contextual cues without retaining them over time.

- **Long-term memory**, on the other hand, enables the agent to retain and recall vast amounts of information over extended periods. This is often facilitated by an external memory module or vector store, which helps the agent access and retrieve relevant past experiences efficiently.

In addition to these memory types, an agent's system also incorporates a reflection mechanism that synthesizes past experiences into higher-level inferences, guiding future behavior based on learned lessons. The effectiveness of memory in this context is influenced by factors such as recency, importance, and relevance of past events to current tasks or queries.

Overall, these m